### Importing packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Exploratory Data Analysis

In [25]:
delivery_data = pd.read_csv("parcel_delivery_dataset.csv")

delivery_data.head()

,parcel_id,shipment_date,is_weekend,hour,distance_km,parcel_size,parcel_weight_kg,traffic_level,weather_condition,delivery_time_window
0,1,2024-04-17,0,14,38.079472,small,3.330077,low,foggy,afternoon
1,2,2024-07-07,1,14,95.120716,medium,0.675873,low,rainy,afternoon
2,3,2024-03-27,0,14,73.467400,medium,45.398848,high,sunny,evening
3,4,2024-04-29,0,14,60.267190,medium,28.637023,low,sunny,evening
4,5,2024-06-06,0,9,16.445845,small,9.344217,high,snowy,evening


In [26]:
delivery_data.dtypes

parcel_id                 int64
shipment_date            object
is_weekend                int64
hour                      int64
distance_km             float64
parcel_size              object
parcel_weight_kg        float64
traffic_level            object
weather_condition        object
delivery_time_window     object
dtype: object

In [27]:
duplicate_rows = delivery_data[delivery_data.duplicated()]

print(f"There are {duplicate_rows.shape[0]} duplicate rows.")

There are 0 duplicate rows.


In [28]:
rows_with_nulls = delivery_data.isnull().sum()
rows_with_nulls

parcel_id               0
shipment_date           0
is_weekend              0
hour                    0
distance_km             0
parcel_size             0
parcel_weight_kg        0
traffic_level           0
weather_condition       0
delivery_time_window    0
dtype: int64

So there are no duplicate rows in the dataset and no rows containing any NaN or Null values.